# Pybind11 test for EjfatURI class

First import the library "e2sar_py". Make sure the module is included in the path.

In [1]:
import sys

# IMPORTANT: Adjust the path to your built Python module as necessary
sys.path.append(
    '/home/xmei/Documents/hpdf_projects/E2SAR/build/src/pybind')

import e2sar_py

print(e2sar_py.get_version())
# Print to check the attributes
# print(dir(e2sar_py))

0.1.0a6


(*Optional*) Test the Python `IPaddress` class.

In [2]:
ip_instance = e2sar_py.IPAddress.from_string("192.168.1.3")
print(ip_instance)
print(ip_instance.is_v4())
print(ip_instance.is_v6())

192.168.1.3
True
False


In [3]:
DEMO_URI = "ejfat://token@192.188.29.6:18020/lb/36?sync=192.188.29.6:19020&data=192.188.29.20"

In [4]:
uri = e2sar_py.EjfatURI(DEMO_URI)

print(dir(uri))   # print the supported methods


['TokenType', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'admin', 'get_admin_token', 'get_cp_addr', 'get_cp_host', 'get_data_addr_v4', 'get_data_addr_v6', 'get_from_env', 'get_from_file', 'get_from_string', 'get_instance_token', 'get_session_token', 'get_sync_addr', 'get_use_tls', 'has_data_addr', 'has_data_addr_v4', 'has_data_addr_v6', 'has_sync_addr', 'instance', 'lb_id', 'lb_name', 'session', 'session_id', 'set_data_addr', 'set_instance_token', 'set_session_token', 'set_sync_addr', 'to_string']


In [5]:

print(uri.get_use_tls())

False


For C++ class `EjfatURI`, private variables `lbName` and `lbId` are mapped as Python attributes `EjfatURI.lb_name` and `EjfatURI.lb_id`. Their C++ get/set methods are simplified.

In [6]:
if (uri.lb_name):
    print(uri.lb_name)
else:
    print("LB name not set!")

uri.lb_name = "lb_pytest" # equal to Ejfat::set_LbName
print(uri.lb_name)  # equal to Ejfat::get_LbName

LB name not set!
lb_pytest


`EjfatURI::set_dataAddr()` and `Ejfat::set_syncAddr()` related bindings.

In [7]:
# Set syncAddr
sync_addr = (e2sar_py.IPAddress.from_string("192.168.1.1"), 8080)
uri.set_sync_addr(sync_addr)

# Set dataAddr
data_addr_v4 = (e2sar_py.IPAddress.from_string("10.0.0.1"), 9090)
uri.set_data_addr(data_addr_v4)

print(uri.has_data_addr())
print(uri.has_sync_addr())
print(uri.has_data_addr_v4())
print(uri.has_data_addr_v6())

True
True
True
False


Bindings where the return type is `result<std::string>`.

In [8]:
def get_inst_token(uri : e2sar_py.EjfatURI):
    try:
        token = uri.get_instance_token().value()
        print("Instance Token:", token)
    except RuntimeError as e:
        print("Instance Token - Error:", e)

get_inst_token(uri)  # should print error

uri.set_instance_token("token123")
get_inst_token(uri)  # print the token value



Instance Token - Error: Instance token not available
Instance Token: token123


Bindings where the return type is `std::pair<ip::address, uint16_t>`.

In [9]:
# Get control plane address
try:
    cp_addr = uri.get_cp_addr().value()
    print("Control Plane Address:", str(cp_addr[0]))
    print("Control Plane Port:", cp_addr[1])
except RuntimeError as e:
    print("Control Plane Address - Error:", e)

Control Plane Address: 192.188.29.6
Control Plane Port: 18020


In [10]:
# Get data plane IPv4 address
print("Data Plane Address (v4):", str(uri.get_data_addr_v4().value()[0]))

# Get data plane IPv6 address
try:
    data_addr_v6 = uri.get_data_addr_v6().value()
    print("Data Plane Address (v6):", str(data_addr_v6[0]))
except RuntimeError as e:
    print("Data Plane Address (v6) - Error:", e)

# Get sync address
print("Sync Address:", str(uri.get_sync_addr().value()[0]))

Data Plane Address (v4): 10.0.0.1
Data Plane Address (v6) - Error: Data plane address not available
Sync Address: 192.168.1.1


C++ return type is `result<std::pair<std::string, u_int16_t>>`.

In [11]:
# Control plane host
try:
    cp_host = uri.get_cp_host().value()
    print("Control Plane Host:", str(cp_host[0]))
    print("Control Plane Host Port:", cp_host[1])
except RuntimeError as e:
    print("Control Plane Host - Error:", e)

Control Plane Host - Error: Control plane hostname not available
